In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
from cryocat import cryomotl
from cryocat import motl_conversions as mc
from cryocat import ribana as ra

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
motls = np.array([
    'j010run_data.star',
    'j056run_data.star',
    'j066run_data.star',
    'j069run_data.star',
    'j070run_data.star',
    'j076run_data.star',
    'j108run_data.star',
    'j120run_data.star',
    'j449run_data.star',    # the original was 'j122run_data.star',
    'j148run_data.star',
    'j155run_data.star',
    'j166run_data.star',
    'j167run_data.star',
    'j172run_data.star',
    'j961run_data.star',    # the original was 'j196run_data.star'
    'j796run_data.star',    # the original was 'j201run_data.star'
    'j424run_data.star',
    'j627run_data.star'
])

# Tomo dimensions - same for all tomograms in dataset, bin10 has 410x410x300
dim=np.array([[2050,2050,1500]])

for m in motls:
    input_path='./motls/' + m
    output_path='./motls/' + m[:-4] + 'em'
    
    # Convert relion to motl, pixel size at bin1 is 1.223, bin2 is 2.446
    nm=mc.relion2emmotl(input_path,pixel_size=2.446)
    
    # Handedness flip
    nm.flip_handedness(dim)

    # Updating centers
    nm.update_coordinates()

    # Writing out
    nm.write_to_emfile(output_path)

In [ ]:
# New classification
motls_u = [ './motls/j167run_data.em',      # cl 1
            './motls/j166run_data.em',      # cl 2
            './motls/j148run_data.em',      # cl 3
            './motls/j796run_data.em',      # cl 4
            './motls/j172run_data.em',      # cl 5
            './motls/j961run_data.em',      # cl 6
            './motls/j627run_data.em',      # cl 7
            './motls/j155run_data.em']      # cl 8

# 108 and 120 (previously classes 10 and 11) were merged into 424 which corresponds to the class 5 now
motls_t = [ './motls/j424run_data.em',                  # cl 5 
            './motls/j449run_data.em',                  # cl 6
            './motls/j076run_data.em',                  # cl 7
            './motls/j070run_data.em',                  # cl 8
            './motls/j069run_data.em',                  # cl 9
            './motls/j066run_data.em']                  # cl 10

In [ ]:
classified_motl_u = ra.assign_class(
    './motls/j056run_data_dc40.em', 
    motls_u, 
    output_motl = './motl_j056_classified_bin2.em', 
    starting_class = 1, 
    dist_threshold = 20,    # in pixels -> 20 * 0.2446, roughly 5nm
    unassigned_class = 0)

In [ ]:
classified_motl_t = ra.assign_class(
    './motls/j010run_data_dc40.em', 
    motls_t, 
    output_motl = './motl_j010_classified_bin2.em', 
    starting_class = 5, 
    dist_threshold = 20, 
    unassigned_class = 0)

In [ ]:
sh_motl_u_entry = cryomotl.Motl.recenter_subparticle(['motl_j056_classified_bin2.em'],['entry_mask_vop_j056_righthand.mrc'],[48])
sh_motl_u_exit = cryomotl.Motl.recenter_subparticle(['motl_j056_classified_bin2.em'],['exit_mask_vop_j056_righthand.mrc'],[48])
sh_motl_u_entry.write_to_emfile('entry_motl_j056_bin2.em')
sh_motl_u_exit.write_to_emfile('exit_motl_j056_bin2.em')

sh_motl_t_entry = cryomotl.Motl.recenter_subparticle(['motl_j010_classified_bin2.em'],['entry_mask_vop_j010_righthand.mrc'],[48])
sh_motl_t_exit = cryomotl.Motl.recenter_subparticle(['motl_j010_classified_bin2.em'],['exit_mask_vop_j010_righthand.mrc'],[48])
sh_motl_t_entry.write_to_emfile('entry_motl_j010_bin2.em')
sh_motl_t_exit.write_to_emfile('exit_motl_j010_bin2.em')


In [ ]:
motl_entry = 'entry_motl_j056_bin2.em'
motl_exit = 'exit_motl_j056_bin2.em'
pixel_size = 0.2446 # in nm

for i in np.arange(1, 26, 1):

    min_d = (i-0.5)/pixel_size 
    max_d = (i+0.5)/pixel_size 
    
    output_name = './traced_min_max_0.5/tmotl_j056_' + str(i) + 'nm_bin2.em'
    ra.trace_chains(motl_entry,motl_exit,max_distance=max_d, min_distance=min_d, output_motl=output_name)

motl_entry = 'entry_motl_j010_bin2.em'
motl_exit = 'exit_motl_j010_bin2.em'
pixel_size = 0.2446 # in nm

for i in np.arange(1, 26, 1):

    min_d = (i-0.5)/pixel_size 
    max_d = (i+0.5)/pixel_size 
    
    output_name = './traced_min_max_0.5/tmotl_j010_' + str(i) + 'nm_bin2.em'
    ra.trace_chains(motl_entry,motl_exit,max_distance=max_d, min_distance=min_d, output_motl=output_name)

In [ ]:
motl_entry = 'entry_motl_j056_bin2.em'
motl_exit = 'exit_motl_j056_bin2.em'
pixel_size = 0.2446 # in nm

max_distance=9/pixel_size
traced_motl=ra.trace_chains(motl_entry,motl_exit,max_distance=max_distance, min_distance=0)
traced_motl.df.sort_values(['tomo_id','object_id', 'geom2'], inplace=True)

ra.add_occupancy(traced_motl)
ra.add_traced_info(traced_motl, motl_entry, 'entry_motl_j056_bin2_tr9nm.em')
ra.add_traced_info(traced_motl, motl_exit, 'exit_motl_j056_bin2_tr9nm.em')
ra.add_traced_info(traced_motl,'motl_j056_classified_bin2.em', 'motl_j056_classified_bin2_tr9nm.em')


In [ ]:
motl_entry = 'entry_motl_j010_bin2.em'
motl_exit = 'exit_motl_j010_bin2.em'
pixel_size = 0.2446 # in nm

max_distance=9/pixel_size
traced_motl=ra.trace_chains(motl_entry,motl_exit,max_distance=max_distance, min_distance=0)
traced_motl.df.sort_values(['tomo_id','object_id', 'geom2'], inplace=True)

ra.add_occupancy(traced_motl)
ra.add_traced_info(traced_motl, motl_entry, 'entry_motl_j010_bin2_tr9nm.em')
ra.add_traced_info(traced_motl, motl_exit, 'exit_motl_j010_bin2_tr9nm.em')
ra.add_traced_info(traced_motl,'motl_j010_classified_bin2.em', 'motl_j010_classified_bin2_tr9nm.em')